In [1]:
import tensorflow as tf
import pandas as pd
import sys
sys.path.append('../')
from config import *
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.preprocessing import sequence
import numpy as np
from tqdm import tqdm
from IPython.display import clear_output

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
tf.enable_eager_execution()
tf.test.is_gpu_available()

True

In [ ]:
### 占用gpu
a = tf.constant(1)
res = tf.add(a, a)
sess = tf.Session()
sess.run(res)

In [97]:
### Hyperparameter
###################
SAVE_PATH = '/home/huangzc/competition/tencent/model_ckpt/simple_nn/model.ckpt'
num_layers = 4
d_model = 128
dff = 512
num_heads = 8

input_vocab_size = tokenizer_pt.vocab_size + 2
target_vocab_size = tokenizer_en.vocab_size + 2
dropout_rate = 0.1

In [13]:
tr_log = pd.read_pickle(TRAIN_DIR+USER_LOG_PATH)
ts_log = pd.read_pickle(TEST_DIR+USER_LOG_PATH)

tr_ad_df = pd.read_pickle(TRAIN_DIR+AD_INFO_PATH)
ts_ad_df = pd.read_pickle(TEST_DIR+AD_INFO_PATH)

assert tr_log['creative_id'].values.tolist() == tr_ad_df['creative_id'].values.tolist()
assert ts_log['creative_id'].values.tolist() == ts_ad_df['creative_id'].values.tolist()

tr_ad_id_log = pd.concat([tr_log, tr_ad_df], axis=1)
ts_ad_id_log = pd.concat([ts_log, ts_ad_df], axis=1)

tr_ad_id_log = tr_ad_id_log[['user_id', 'age', 'gender', 'click_times', 'ad_id']]
ts_ad_id_log = ts_ad_id_log[['user_id', 'click_times', 'ad_id']]

In [14]:
#读取
f = open(TRAIN_DIR+'gensim_ad_id_dict.js','r')
a = f.read()
vocab_dict = eval(a)
f.close()
wv_matrix = np.load(TRAIN_DIR+'gensim_ad_id.npy')
vocab = vocab_dict.keys()
vocab_size = len(vocab)

In [16]:
tr_ad_id_log['ad_id'] = tr_ad_id_log['ad_id'].astype(str)
ts_ad_id_log['ad_id'] = ts_ad_id_log['ad_id'].astype(str)

tr_ad_id_log = tr_ad_id_log[tr_ad_id_log['ad_id'].isin(vocab)]
ts_ad_id_log = ts_ad_id_log[ts_ad_id_log['ad_id'].isin(vocab)]

In [17]:
tr_ad_id_log['ad_id_index'] = tr_ad_id_log['ad_id'].map(lambda x: vocab_dict[x])
ts_ad_id_log['ad_id_index'] = ts_ad_id_log['ad_id'].map(lambda x: vocab_dict[x])

In [18]:
def get_clk_list(df):
    return list(df.values)

In [19]:
grid_df = tr_ad_id_log.groupby(['user_id']).agg({'age': 'first', 'gender': 'first', 'click_times': lambda x: get_clk_list(x), 'ad_id_index': lambda x: get_clk_list(x)}).reset_index()
grid_df_ts = ts_ad_id_log.groupby(['user_id']).agg({'ad_id_index': lambda x: get_clk_list(x)}).reset_index()

In [21]:
feature_columns = ['click_times', 'ad_id_index']
TARGET = 'age'

grid_df[TARGET] = grid_df[TARGET] - 1

In [ ]:
### Split into train and valid dataset
train = grid_df[feature_columns+[TARGET]]
test = grid_df_ts[feature_columns]

train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

### x, y dataframe
x_train, y_train = train[feature_columns], train[TARGET]
x_val, y_val = val[feature_columns], val[TARGET]
x_test = test

In [ ]:
sentence_size = int(grid_df[feature_columns[0]].map(lambda x: len(x)).quantile(0.9))
print('choose sentences max len: %d' % sentence_size)

In [106]:
print("Pad sequences (samples x time)")

### feature_columns[0] means click list
x_train = sequence.pad_sequences(x_train[feature_columns[0]],
                                 maxlen=sentence_size, 
                                 padding='post', 
                                 truncating='post',
                                 dtype='int64',
                                 value=float(vocab_size)
                                 )
x_val = sequence.pad_sequences(x_val[feature_columns[0]],
                                 maxlen=sentence_size, 
                                 padding='post', 
                                 truncating='post',
                                 dtype='int64',
                                 value=float(vocab_size)
                                 )
x_test = sequence.pad_sequences(x_test[feature_columns[0]], 
                                maxlen=sentence_size, 
                                padding='post',
                                truncating='post',
                                dtype='int64',
                                value=float(vocab_size)
                               )
print("x_train shape:", x_train.shape)
print("x_val shape:", x_val.shape)
print("x_test shape:", x_test.shape)

Pad sequences (samples x time)
x_train shape: (720000, 65)
x_val shape: (180000, 65)
x_test shape: (1000000, 65)


In [107]:
### Get tf dataset
def get_train_ds(x, y): 
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=len(x))
    dataset = dataset.batch(BATCH_SIZE)
#     dataset = dataset.repeat(EPOCHS)
    return dataset

def get_test_ds(x, ): 
    dataset = tf.data.Dataset.from_tensor_slices((x, )).batch(BATCH_SIZE)
    return dataset

train_ds = get_train_ds(x_train, y_train.values)
valid_ds = get_train_ds(x_val, y_val.values)
test_ds = get_test_ds(x_test)

In [150]:
weights = np.random.rand(1, wv_matrix.shape[1])
weights = np.concatenate([wv_matrix, weights], axis=0)

In [151]:
### Construct Model
#################################
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size+1, EMB_SIZE, weights=[weights])
#         self.embedding = tf.keras.layers.Embedding(vocab_size, EMB_SIZE)
        self.pool1D = tf.keras.layers.GlobalAveragePooling1D()
        self.dense1 = tf.keras.layers.Dense(16, activation=tf.nn.leaky_relu)
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.pool1D(x)
        x = self.dense1(x)
        return self.dense2(x)

model = MyModel()

In [152]:
# Restore the weights
# model.load_weights(SAVE_PATH)

In [153]:
### Loss & Metric
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

valid_loss = tf.keras.metrics.Mean(name='valid_loss')
valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_accuracy')

In [154]:
@tf.function
def train_step(features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [155]:
@tf.function
def valid_step(features, labels):
    predictions = model(features)
    v_loss = loss_object(labels, predictions)

    valid_loss(v_loss)
    valid_accuracy(labels, predictions)

In [156]:
for epoch in range(EPOCHS):
  # 在下一个epoch开始时，重置评估指标
    train_loss.reset_states()
    train_accuracy.reset_states()
    valid_loss.reset_states()
    valid_accuracy.reset_states()

    CNT = 0
    for features, labels in train_ds:
#         clear_output(wait=True)
        train_step(features, labels)
        CNT += 1
        print('batch: %d, batch loss: %.4f' % (CNT, train_loss.result()))    
        
    for val_features, val_labels in valid_ds:
        valid_step(val_features, val_labels)

    template = 'Epoch {}, Loss: {:.2f}, Accuracy: {:.2f}%, Valid Loss: {:.2f}, Valid Accuracy: {:.2f}%'
    print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         valid_loss.result(),
                         valid_accuracy.result()*100))

batch: 1, batch loss: 2.3165
batch: 2, batch loss: 2.3149
batch: 3, batch loss: 2.3095
batch: 4, batch loss: 2.3064
batch: 5, batch loss: 2.3021
batch: 6, batch loss: 2.2980
batch: 7, batch loss: 2.2946
batch: 8, batch loss: 2.2913
batch: 9, batch loss: 2.2865
batch: 10, batch loss: 2.2834
batch: 11, batch loss: 2.2791
batch: 12, batch loss: 2.2755
batch: 13, batch loss: 2.2719
batch: 14, batch loss: 2.2687
batch: 15, batch loss: 2.2652
batch: 16, batch loss: 2.2617
batch: 17, batch loss: 2.2580
batch: 18, batch loss: 2.2546
batch: 19, batch loss: 2.2509
batch: 20, batch loss: 2.2478
batch: 21, batch loss: 2.2444
batch: 22, batch loss: 2.2413
batch: 23, batch loss: 2.2377
batch: 24, batch loss: 2.2344
batch: 25, batch loss: 2.2311
batch: 26, batch loss: 2.2279
batch: 27, batch loss: 2.2247
batch: 28, batch loss: 2.2212
batch: 29, batch loss: 2.2182
batch: 30, batch loss: 2.2148
batch: 31, batch loss: 2.2119
batch: 32, batch loss: 2.2086
batch: 33, batch loss: 2.2056
batch: 34, batch lo

batch: 55, batch loss: 1.6403
batch: 56, batch loss: 1.6398
batch: 57, batch loss: 1.6393
batch: 58, batch loss: 1.6389
batch: 59, batch loss: 1.6385
batch: 60, batch loss: 1.6381
batch: 61, batch loss: 1.6376
batch: 62, batch loss: 1.6371
batch: 63, batch loss: 1.6366
batch: 64, batch loss: 1.6360
batch: 65, batch loss: 1.6354
batch: 66, batch loss: 1.6352
batch: 67, batch loss: 1.6349
batch: 68, batch loss: 1.6344
batch: 69, batch loss: 1.6339
batch: 70, batch loss: 1.6336
batch: 71, batch loss: 1.6330
Epoch 4, Loss: 1.63, Accuracy: 34.58%, Valid Loss: 1.60, Valid Accuracy: 35.24%
batch: 1, batch loss: 1.6093
batch: 2, batch loss: 1.6023
batch: 3, batch loss: 1.5948
batch: 4, batch loss: 1.5943
batch: 5, batch loss: 1.5942
batch: 6, batch loss: 1.5941
batch: 7, batch loss: 1.5929
batch: 8, batch loss: 1.5920
batch: 9, batch loss: 1.5906
batch: 10, batch loss: 1.5921
batch: 11, batch loss: 1.5919
batch: 12, batch loss: 1.5915
batch: 13, batch loss: 1.5915
batch: 14, batch loss: 1.5912

batch: 35, batch loss: 1.5093
batch: 36, batch loss: 1.5091
batch: 37, batch loss: 1.5095
batch: 38, batch loss: 1.5095
batch: 39, batch loss: 1.5093
batch: 40, batch loss: 1.5092
batch: 41, batch loss: 1.5091
batch: 42, batch loss: 1.5093
batch: 43, batch loss: 1.5091
batch: 44, batch loss: 1.5089
batch: 45, batch loss: 1.5087
batch: 46, batch loss: 1.5087
batch: 47, batch loss: 1.5086
batch: 48, batch loss: 1.5084
batch: 49, batch loss: 1.5084
batch: 50, batch loss: 1.5082
batch: 51, batch loss: 1.5082
batch: 52, batch loss: 1.5083
batch: 53, batch loss: 1.5081
batch: 54, batch loss: 1.5079
batch: 55, batch loss: 1.5079
batch: 56, batch loss: 1.5078
batch: 57, batch loss: 1.5076
batch: 58, batch loss: 1.5077
batch: 59, batch loss: 1.5077
batch: 60, batch loss: 1.5077
batch: 61, batch loss: 1.5074
batch: 62, batch loss: 1.5074
batch: 63, batch loss: 1.5073
batch: 64, batch loss: 1.5070
batch: 65, batch loss: 1.5068
batch: 66, batch loss: 1.5071
batch: 67, batch loss: 1.5071
batch: 68,

batch: 15, batch loss: 1.4528
batch: 16, batch loss: 1.4531
batch: 17, batch loss: 1.4527
batch: 18, batch loss: 1.4532
batch: 19, batch loss: 1.4528
batch: 20, batch loss: 1.4532
batch: 21, batch loss: 1.4534
batch: 22, batch loss: 1.4534
batch: 23, batch loss: 1.4533
batch: 24, batch loss: 1.4535
batch: 25, batch loss: 1.4535
batch: 26, batch loss: 1.4539
batch: 27, batch loss: 1.4543
batch: 28, batch loss: 1.4537
batch: 29, batch loss: 1.4538
batch: 30, batch loss: 1.4540
batch: 31, batch loss: 1.4540
batch: 32, batch loss: 1.4538
batch: 33, batch loss: 1.4539
batch: 34, batch loss: 1.4538
batch: 35, batch loss: 1.4537
batch: 36, batch loss: 1.4537
batch: 37, batch loss: 1.4542
batch: 38, batch loss: 1.4542
batch: 39, batch loss: 1.4540
batch: 40, batch loss: 1.4540
batch: 41, batch loss: 1.4539
batch: 42, batch loss: 1.4541
batch: 43, batch loss: 1.4539
batch: 44, batch loss: 1.4538
batch: 45, batch loss: 1.4535
batch: 46, batch loss: 1.4536
batch: 47, batch loss: 1.4536
batch: 48,

batch: 68, batch loss: 1.4185
batch: 69, batch loss: 1.4184
batch: 70, batch loss: 1.4185
batch: 71, batch loss: 1.4172
Epoch 15, Loss: 1.42, Accuracy: 41.88%, Valid Loss: 1.48, Valid Accuracy: 39.47%
batch: 1, batch loss: 1.4158
batch: 2, batch loss: 1.4115
batch: 3, batch loss: 1.4061
batch: 4, batch loss: 1.4062
batch: 5, batch loss: 1.4063
batch: 6, batch loss: 1.4062
batch: 7, batch loss: 1.4057
batch: 8, batch loss: 1.4049
batch: 9, batch loss: 1.4035
batch: 10, batch loss: 1.4050
batch: 11, batch loss: 1.4061
batch: 12, batch loss: 1.4061
batch: 13, batch loss: 1.4060
batch: 14, batch loss: 1.4063
batch: 15, batch loss: 1.4070
batch: 16, batch loss: 1.4074
batch: 17, batch loss: 1.4068
batch: 18, batch loss: 1.4075
batch: 19, batch loss: 1.4070
batch: 20, batch loss: 1.4073
batch: 21, batch loss: 1.4075
batch: 22, batch loss: 1.4075
batch: 23, batch loss: 1.4075
batch: 24, batch loss: 1.4077
batch: 25, batch loss: 1.4077
batch: 26, batch loss: 1.4081
batch: 27, batch loss: 1.408

...

batch: 1, batch loss: 1.0762
batch: 2, batch loss: 1.0720
batch: 3, batch loss: 1.0698
batch: 4, batch loss: 1.0734
batch: 5, batch loss: 1.0736
batch: 6, batch loss: 1.0727
batch: 7, batch loss: 1.0712
batch: 8, batch loss: 1.0709
batch: 9, batch loss: 1.0688
batch: 10, batch loss: 1.0699
batch: 11, batch loss: 1.0704
batch: 12, batch loss: 1.0702
batch: 13, batch loss: 1.0699
batch: 14, batch loss: 1.0698
batch: 15, batch loss: 1.0707
batch: 16, batch loss: 1.0716
batch: 17, batch loss: 1.0707
batch: 18, batch loss: 1.0711
batch: 19, batch loss: 1.0707
batch: 20, batch loss: 1.0706
batch: 21, batch loss: 1.0709
batch: 22, batch loss: 1.0706
batch: 23, batch loss: 1.0708
batch: 24, batch loss: 1.0710
batch: 25, batch loss: 1.0711
batch: 26, batch loss: 1.0711
batch: 27, batch loss: 1.0715
batch: 28, batch loss: 1.0710
batch: 29, batch loss: 1.0710
batch: 30, batch loss: 1.0714
batch: 31, batch loss: 1.0713
batch: 32, batch loss: 1.0711
batch: 33, batch loss: 1.0712
batch: 34, batch lo

batch: 55, batch loss: 1.0184
batch: 56, batch loss: 1.0184
batch: 57, batch loss: 1.0183
batch: 58, batch loss: 1.0184
batch: 59, batch loss: 1.0183
batch: 60, batch loss: 1.0184
batch: 61, batch loss: 1.0183
batch: 62, batch loss: 1.0182
batch: 63, batch loss: 1.0182
batch: 64, batch loss: 1.0180
batch: 65, batch loss: 1.0178
batch: 66, batch loss: 1.0181
batch: 67, batch loss: 1.0182
batch: 68, batch loss: 1.0180
batch: 69, batch loss: 1.0180
batch: 70, batch loss: 1.0178
batch: 71, batch loss: 1.0134
Epoch 45, Loss: 1.01, Accuracy: 59.91%, Valid Loss: 1.63, Valid Accuracy: 40.28%
batch: 1, batch loss: 1.0066
batch: 2, batch loss: 1.0027
batch: 3, batch loss: 1.0008
batch: 4, batch loss: 1.0051
batch: 5, batch loss: 1.0054
batch: 6, batch loss: 1.0044
batch: 7, batch loss: 1.0027
batch: 8, batch loss: 1.0026
batch: 9, batch loss: 1.0005
batch: 10, batch loss: 1.0016
batch: 11, batch loss: 1.0021
batch: 12, batch loss: 1.0018
batch: 13, batch loss: 1.0015
batch: 14, batch loss: 1.001

batch: 35, batch loss: 0.9473
batch: 36, batch loss: 0.9473
batch: 37, batch loss: 0.9476
batch: 38, batch loss: 0.9478
batch: 39, batch loss: 0.9476
batch: 40, batch loss: 0.9474
batch: 41, batch loss: 0.9474
batch: 42, batch loss: 0.9475
batch: 43, batch loss: 0.9473
batch: 44, batch loss: 0.9472
batch: 45, batch loss: 0.9469
batch: 46, batch loss: 0.9469
batch: 47, batch loss: 0.9468
batch: 48, batch loss: 0.9467
batch: 49, batch loss: 0.9468
batch: 50, batch loss: 0.9466
batch: 51, batch loss: 0.9464
batch: 52, batch loss: 0.9465
batch: 53, batch loss: 0.9463
batch: 54, batch loss: 0.9460
batch: 55, batch loss: 0.9460
batch: 56, batch loss: 0.9459
batch: 57, batch loss: 0.9459
batch: 58, batch loss: 0.9459
batch: 59, batch loss: 0.9459
batch: 60, batch loss: 0.9460
batch: 61, batch loss: 0.9459
batch: 62, batch loss: 0.9458
batch: 63, batch loss: 0.9458
batch: 64, batch loss: 0.9455
batch: 65, batch loss: 0.9453
batch: 66, batch loss: 0.9456
batch: 67, batch loss: 0.9457
batch: 68,

batch: 15, batch loss: 0.8727
batch: 16, batch loss: 0.8736
batch: 17, batch loss: 0.8727
batch: 18, batch loss: 0.8727
batch: 19, batch loss: 0.8724
batch: 20, batch loss: 0.8722
batch: 21, batch loss: 0.8724
batch: 22, batch loss: 0.8723
batch: 23, batch loss: 0.8725
batch: 24, batch loss: 0.8727
batch: 25, batch loss: 0.8727
batch: 26, batch loss: 0.8727
batch: 27, batch loss: 0.8729
batch: 28, batch loss: 0.8724
batch: 29, batch loss: 0.8725
batch: 30, batch loss: 0.8727
batch: 31, batch loss: 0.8725
batch: 32, batch loss: 0.8723
batch: 33, batch loss: 0.8723
batch: 34, batch loss: 0.8718
batch: 35, batch loss: 0.8716
batch: 36, batch loss: 0.8716
batch: 37, batch loss: 0.8718
batch: 38, batch loss: 0.8720
batch: 39, batch loss: 0.8719
batch: 40, batch loss: 0.8717
batch: 41, batch loss: 0.8717
batch: 42, batch loss: 0.8717
batch: 43, batch loss: 0.8715
batch: 44, batch loss: 0.8715
batch: 45, batch loss: 0.8711
batch: 46, batch loss: 0.8712
batch: 47, batch loss: 0.8710
batch: 48,

KeyboardInterrupt: 

win:10, 39.78 <br>
win: 2, 39.38 <br>
win:15, 40.18 <br>
win:20, 40.76 <br>
win:30, 40.57

In [ ]:
# 保存权重
model.save_weights(SAVE_PATH)